## BUILD THE NEURAL NETWORK
## 构建神经网络

神经网络由对数据执行操作的层/模块组成。torch.nn命名空间提供了构建自己的神经网络所需的所有构建块。PyTorch中的每个模块都是 nn.Module 的子类。神经网络是一个模块本身，它由其他模块（层）组成。这种嵌套结构允许轻松构建和管理复杂的架构

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training
### 获得训练设备

检查一下 torch.cuda是否可用，否则我们继续使用CPU

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### Define the Class
### 定义类

In [ ]:
"""
通过子类化定义我们的神经网络nn.Module，并在中初始化神经
网络层__init__。每个nn.Module子类都在方法中实现对输入数
据的操作forward
"""

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

创建 的实例NeuralNetwork，并将其移动到device，并打印其结构

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


要使用模型，我们将输入数据传递给它。这将执行模型的forward，以及一些后台操作。不要直接使用model.forward()！

在输入上调用模型会返回一个10维张量，其中包含每个类的原始预测值。通过nn.Softmax模块的一个实例来获得预测概率。

In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[0.0920, 0.1008, 0.1154, 0.1003, 0.0978, 0.0999, 0.0974, 0.0952, 0.1041,
         0.0970]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([2])


### Model Layers
### 模型层

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

初始化nn.Flatten 层以将每个 2D 28x28 图像转换为 784 个像素值的连续数组（保持小批量维度（dim=0））

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

线性层是一个模块，它 使用其存储的权重和偏差对输入应用线性变换

In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


非线性激活是在模型的输入和输出之间创建复杂映射的原因。它们在线性变换后应用以引入非线性，帮助神经网络学习各种现象。

在这个模型中，我们在线性层之间使用nn.ReLU，但是还有其他激活可以在模型中引入非线性

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1941,  0.4187,  0.4639,  0.2990, -0.4311, -0.2235,  0.1667, -0.1878,
          0.2090, -0.2200, -0.0512,  0.2211, -0.0501, -0.1494, -0.1426, -0.1378,
          0.2996,  0.3582, -0.0458,  0.0737],
        [ 0.0376,  0.3173,  0.5275,  0.5480, -0.4904, -0.2354,  0.0687, -0.0312,
          0.0901, -0.4720, -0.1748,  0.6029,  0.1113, -0.1404, -0.0932,  0.1158,
         -0.0167,  0.1138, -0.0973,  0.0955],
        [ 0.1081,  0.3534,  0.4773,  0.4659, -0.3959, -0.3443,  0.0204, -0.2425,
          0.2798, -0.8940, -0.3600,  0.4387,  0.1209, -0.1381, -0.3003,  0.0804,
          0.2476, -0.0169, -0.2428, -0.3173]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.4187, 0.4639, 0.2990, 0.0000, 0.0000, 0.1667, 0.0000, 0.2090,
         0.0000, 0.0000, 0.2211, 0.0000, 0.0000, 0.0000, 0.0000, 0.2996, 0.3582,
         0.0000, 0.0737],
        [0.0376, 0.3173, 0.5275, 0.5480, 0.0000, 0.0000, 0.0687, 0.0000, 0.0901,
         0.0000, 0.0000, 0.6029, 0.1113, 0.0000, 0.00

### nn.Sequential

nn.Sequential是一个有序的模块容器。数据按照定义的顺序通过所有模块。您可以使用顺序容器来组合一个快速网络，例如seq_modules

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

神经网络的最后一个线性层返回logits - [-infty, infty] 中的原始值 - 被传递给 nn.Softmax模块。logits 被缩放为值 [0, 1]，表示模型对每个类别的预测概率。dim参数指示值必须总和为 1 的维度

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
## 模型参数

神经网络内的许多层都是参数化的，即具有在训练期间优化的相关权重和偏差。子类nn.Module化会自动跟踪模型对象中定义的所有字段，并使用模型parameters()或named_parameters()方法使所有参数都可以访问。

在此示例中，我们遍历每个参数，并打印其大小和其值的预览

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0038, -0.0207,  0.0013,  ...,  0.0263, -0.0162,  0.0270],
        [ 0.0308,  0.0055, -0.0062,  ..., -0.0139, -0.0071, -0.0236]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0148, -0.0227], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 4.3909e-02, -3.9007e-02, -1.6041e-05,  ..., -4.0481e-02,
          3.9038e-02, -4.4171e-02],
        [ 2.9929e-02, -1.2821e-02,  9.1028e-03,  ..., -2.2743e-02,
         -3.6408e-02, -3.8682e-02]], gra